# Titanic

## 1. Data preprocessing
Importons les modules nécessaires.

In [58]:
import pandas as pd
import plotly.express as px
import re
import numpy as np

Chargeons les données dans deux dataframes.

In [59]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Quelles valeurs attendons-nous ?**

Le dataset Titanic se compose des colonnes suivantes :

- `PassengerId` : Identifiant unique *(entier)*

- `Survived` : État du passager à l'issue du naufrage *(0 ou 1)*

- `Pclass` : Classe du ticket *(1, 2 ou 3)*

- `Name` : Nom du passager *(chaine de caractères)*

- `Sex` : Sexe du passager *('male' ou 'female')*

- `Age` : Age du passager *(nombre positif entre 0 et 120)*

- `SibSp` : Nombre de frères, sœurs et/ou époux à bord *(entier >= 0)*

- `Parch` : Nombre de parents et/ou enfants à bord *(entier >= 0)*

- `Ticket` : Numéro du ticket *(chaine alphanumérique)*

- `Fare` : Prix payé par le passager *(nombre positif)*

- `Cabin` : Numéro de cabine *(chaine alphanumérique)*

- `Embarked` : Port d'embarquement *('C', 'Q' ou 'S')*

Avant toute chose, vérifions qu'il n'y ait pas de valeurs aberrantes.

In [60]:
def check_data_quality(df):
    print('================================')
    # on inspecte chaque colonne du df
    for column in df.columns:
        # global
        print(f'Colonne : {column}')
        print(f'Type : {df[column].dtype}')
        
        # spécifique à certaines colonnes
        if column == 'PassengerId':
            print(f'Toutes les valeurs sont uniques : {df[column].nunique() == len(df[column])}')

        elif column == 'Age':
            print(f'Âges valides (0-120) : {((df[column].dropna() >= 0) & (df[column].dropna() <= 120)).all()} (en ignorant {df[column].isna().sum()} valeurs NaN)')

        elif column in ['SibSp', 'Parch', 'Fare']:
            print(f'Toutes les valeurs sont >= 0 : {(df[column].dropna() >= 0).all()}')
            
        elif column in ['Survived', 'Pclass', 'Sex', 'Embarked']:
            print(f'Valeurs : {df[column].unique()}')
        
        print('---')

check_data_quality(train_data)
check_data_quality(test_data)

Colonne : PassengerId
Type : int64
Toutes les valeurs sont uniques : True
---
Colonne : Survived
Type : int64
Valeurs : [0 1]
---
Colonne : Pclass
Type : int64
Valeurs : [3 1 2]
---
Colonne : Name
Type : object
---
Colonne : Sex
Type : object
Valeurs : ['male' 'female']
---
Colonne : Age
Type : float64
Âges valides (0-120) : True (en ignorant 177 valeurs NaN)
---
Colonne : SibSp
Type : int64
Toutes les valeurs sont >= 0 : True
---
Colonne : Parch
Type : int64
Toutes les valeurs sont >= 0 : True
---
Colonne : Ticket
Type : object
---
Colonne : Fare
Type : float64
Toutes les valeurs sont >= 0 : True
---
Colonne : Cabin
Type : object
---
Colonne : Embarked
Type : object
Valeurs : ['S' 'C' 'Q' nan]
---
Colonne : PassengerId
Type : int64
Toutes les valeurs sont uniques : True
---
Colonne : Pclass
Type : int64
Valeurs : [3 2 1]
---
Colonne : Name
Type : object
---
Colonne : Sex
Type : object
Valeurs : ['male' 'female']
---
Colonne : Age
Type : float64
Âges valides (0-120) : True (en ignorant

Nous pouvons voir qu'aucune valeur inattendue ne se trouve dans nos dataframes.

Cependant, nous remarquons que certaines colonnes ont des types `object`. Nous allons les convertir vers des types plus adaptés.

In [61]:
# convertir certains types

def convert_types(df):
    data_converted = df.copy()

    # CATEGORY: type pandas (liste finie de valeurs texte)

    data_converted['Sex'] = data_converted['Sex'].astype('category')
    data_converted['Embarked'] = data_converted['Embarked'].astype('category')

    # STRING

    data_converted['Name'] = data_converted['Name'].astype('string')
    data_converted['Ticket'] = data_converted['Ticket'].astype('string')
    data_converted['Cabin'] = data_converted['Cabin'].astype('string')

    return data_converted

train_data = convert_types(train_data)
print('success')


success


In [62]:
check_data_quality(train_data)

Colonne : PassengerId
Type : int64
Toutes les valeurs sont uniques : True
---
Colonne : Survived
Type : int64
Valeurs : [0 1]
---
Colonne : Pclass
Type : int64
Valeurs : [3 1 2]
---
Colonne : Name
Type : string
---
Colonne : Sex
Type : category
Valeurs : ['male', 'female']
Categories (2, object): ['female', 'male']
---
Colonne : Age
Type : float64
Âges valides (0-120) : True (en ignorant 177 valeurs NaN)
---
Colonne : SibSp
Type : int64
Toutes les valeurs sont >= 0 : True
---
Colonne : Parch
Type : int64
Toutes les valeurs sont >= 0 : True
---
Colonne : Ticket
Type : string
---
Colonne : Fare
Type : float64
Toutes les valeurs sont >= 0 : True
---
Colonne : Cabin
Type : string
---
Colonne : Embarked
Type : category
Valeurs : ['S', 'C', 'Q', NaN]
Categories (3, object): ['C', 'Q', 'S']
---


Identifions maintenant les données manquantes.

In [63]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**Valeurs manquantes par colonne**

`Age` : 177 valeurs manquantes

`Cabin` : 687 valeurs manquantes

`Embarked` : 2 valeurs manquantes

**Peut-on remplacer les valeurs vides ?**

Nous pourrions remplacer les valeurs `Age` manquantes par la moyenne du profil type. Imaginons quelques-uns de ces groupes :

- `femmes ayant un ticket en première classe`

- `hommes ayant un ticket en troisième classe`

On peut imaginer que le remplacement par la moyenne d'âge de ces groupes serait plus pertinente que la moyenne globale.

Vérifions-le.

In [64]:
# vérifier que c'est plus pertinent que la moyenne globale
age_by_group = train_data.groupby(['Sex', 'Pclass'], observed=False)['Age'].agg(['mean', 'min', 'max', 'count'])

age_by_group

mean   min   max  count
Sex    Pclass                              
female 1       34.611765  2.00  63.0     85
       2       28.722973  2.00  57.0     74
       3       21.750000  0.75  63.0    102
male   1       41.281386  0.92  80.0    101
       2       30.740707  0.67  70.0     99
       3       26.507589  0.42  74.0    253

On observe que les groupes ont bien des moyennes d'`Age` différentes ce qui laisse à penser que c'est une décision qui amenera plus de précision.

Remplaçons maintenant les données manquantes.

In [65]:
# fonction afin de pouvoir reproduire cette étape plus facilement dans la suite de ce projet
def fill_age(data):
    data_filled = data.copy()

    # fonction a utiliser dans transform, on aurait pu utiliser une lambda directement dans transform
    def fillna_with_mean(x):
        return x.fillna(x.mean())

    # remplacer valeurs vides par moyenne du groupe (basé sur sexe + classe)
    data_filled['Age'] = data_filled.groupby(['Sex', 'Pclass'], observed=False)['Age'].transform(fillna_with_mean)

    # si il reste des valeurs vides > utiliser la moyenne globale en dernier recours
    data_filled['Age'] = data_filled['Age'].fillna(data_filled['Age'].mean())

    return data_filled

train_data_age_filled = fill_age(train_data)
test_data_age_filled = fill_age(test_data)

train_data_age_filled.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Nous n'avons plus de valeurs `Age` vides

Il reste, cependant, des valeurs vides dans la colonne `Cabin` qui peut être un indicateur important comme elle représente des niveaux du navire qui pourraient avoir un impact sur la survie.

On remarque que ces valeurs vides concernent `687` lignes ce qui veut dire qu'il est hors de question de les supprimer, d'autant plus que cela peut être une information à part entière. On peut penser qu'il serait plus intéressant de les marquer comme `Unknown`.

In [66]:
def fill_cabin(data):
    data_filled = data.copy()

    data_filled['Cabin'] = data_filled['Cabin'].fillna('Unknown')

    return data_filled

train_data_age_cabin_filled = fill_cabin(train_data_age_filled)
test_data_age_cabin_filled = fill_cabin(test_data_age_filled)

train_data_age_cabin_filled.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64

On n'a plus aucune valeur vides dans la colonne `Cabin` mais il reste `2` infos manquantes dans la colonne `Embarked`.

In [67]:
train_data_age_cabin_filled['Embarked'].value_counts(dropna=False)

Embarked
S      644
C      168
Q       77
NaN      2
Name: count, dtype: int64

Une bonne approche pourrait être de remplir les valeurs manquantes par le port le plus commun en fonction de leur classe.

In [68]:
def fill_embarked(data):
    data_filled = data.copy()

    # loop à travers les lignes où embarked est vide
    for i in data_filled[data_filled['Embarked'].isna()].index:
        # on récupère le pclass de chaque ligne où embarked est vide
        pclass = data_filled.loc[i, 'Pclass']

        # on trouve le port le plus commun pour cette classe
        most_common_port = data_filled[data_filled['Pclass'] == pclass]['Embarked'].mode()[0]
        
        data_filled.loc[i, 'Embarked'] = most_common_port
        print(f'index: {i}, Embarked: {data_filled.loc[i, 'Embarked']}, success')
    
    return data_filled

train_data_filled = fill_embarked(train_data_age_cabin_filled)
test_data_filled = fill_embarked(test_data_age_cabin_filled)

train_data_filled.isna().sum()

index: 61, Embarked: S, success
index: 829, Embarked: S, success


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

Nous n'avons plus aucune valeur manquante dans nos dataframes.

## 2. Features

L'ajout de certaines features peut sembler pertinent :

- Le passager était-il seul ?

- Avait-il une cabine ?

- Sur quel pont se trouvait leur cabine ?

- Quel était son titre ? (Mr, Mrs, Miss...)

In [69]:
# ajout de features
full_data = [train_data_filled, test_data_filled]

# bins
for i in full_data:
    i['AgeBin'] = pd.cut(
        i['Age'],
        bins=[0, 14, 25, 45, 65, np.inf], # bins basés sur le contexte social de l'époque
        labels=['Child', 'Young_Adult', 'Adult', 'Middle_Aged', 'Senior']
    )

    i['FareBin'] = pd.cut(
        i['Fare'],
        bins=[
            0, # 0-8 minimum
            8, # 8-15 3 ème classe
            15, # 15-30 2 ème classe
            30, # 30-80 1 ère classe standard
            80, # 80-infini luxe
            np.inf
        ],
        labels=['Very_Low', 'Low', 'Medium', 'High', 'Very_High']
    )

# is alone
for i in full_data:
    i['IsAlone'] = 0

    # si la condition est true on sélectionne 'IsAlone'
    i.loc[(i['SibSp'] + i['Parch'] == 0), 'IsAlone'] = 1

# has cabin
for i in full_data:
    i['HasCabin'] = 0

    i.loc[i['Cabin'] != 'Unknown', 'HasCabin'] = 1

# cabin deck
for i in full_data:
    i['CabinDeck'] = i['Cabin'].str[0] # Unknown = U

# title from name
def get_title(name):
    # on cherche un espace, on capture la chaine de plusieurs lettres si elle est suivie d'un point (échappé avec le \)
    search_result = re.search(r' ([A-Za-z]+)\.', name) # raw string pour que python ne traite pas l'échappement avant regex

    if search_result:
        # group(0): return ' Mrs.' (TOUT le pattern)
        # group(1): return 'Mrs' (le groupe de capture entre parenthèses)
        # group(2): (le 2eme groupe de capture entre parenthèses s'il existe)
        return search_result.group(1)
    return ''

for i in full_data:
    i['Title'] = i['Name'].apply(get_title)

print(train_data_filled['Title'].value_counts(dropna=False))
print(test_data_filled['Title'].value_counts(dropna=False))
print(train_data_filled['CabinDeck'].value_counts)

train_data_filled.head()

Title
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: count, dtype: int64
Title
Mr        240
Miss       78
Mrs        72
Master     21
Col         2
Rev         2
Ms          1
Dr          1
Dona        1
Name: count, dtype: int64
<bound method IndexOpsMixin.value_counts of 0      U
1      C
2      U
3      C
4      U
      ..
886    U
887    B
888    U
889    C
890    U
Name: CabinDeck, Length: 891, dtype: string>


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeBin,FareBin,IsAlone,HasCabin,CabinDeck,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,Young_Adult,Very_Low,0,0,U,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult,High,0,1,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,Adult,Very_Low,1,0,U,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adult,High,0,1,C,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,Adult,Low,1,0,U,Mr


Nous pouvons voir que nos features ont bien été ajoutées mais dans le cas de la feature `Title` nous avons plusieurs titres très peu utilisés que nous pourrions regrouper sous une seule et même valeur `Rare`.

Nous allons faire cela de manière dynamique en appliquant la modification à tous les titres qui représentent moins de `3 %` de nos dataframes. Et profitons-en pour rassembler les titres similaires tels que `Mlle`, `Ms` = `Miss` et `Mme` = `Mrs`.

In [70]:
# récupérer dynamiquement tous les titles rares pour les unifier dans une même catégorie

# limitation : si on teste peu de données cela risque de laisser passer des valeurs rares
def replace_rare_titles_by_count(full_data = full_data):
    for i in full_data:
        title_count = i['Title'].value_counts()
        percentage = 0.03 # 3 %

        i['Title'] = i['Title'].apply(
            # appliquer le vrai titre s'il en existe plus que 3 % de la len de test_data dans nos données autrement on applique 'Rare'
            lambda real_title: real_title if title_count[real_title] >= (int(len(test_data_filled) * percentage)) else 'Rare' 
            )

    print((len(test_data_filled) * percentage))

def replace_titles(full_data = full_data):
    def check_titles(real_title):
        if real_title in ['Mr', 'Miss', 'Mrs', 'Master']:
            return real_title
        elif real_title in ['Mlle', 'Ms']:
            return 'Miss'
        elif real_title == 'Mme':
            return 'Mrs'
        else:
            return 'Rare'

    for i in full_data:
        i['Title'] = i['Title'].apply(check_titles)

replace_titles()

print(train_data_filled['Title'].value_counts(dropna=False))
print(test_data_filled['Title'].value_counts(dropna=False))

Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64
Title
Mr        240
Miss       79
Mrs        72
Master     21
Rare        6
Name: count, dtype: int64


## 3. Exploration de données

Visualisons nos données sous la forme de plusieurs graphiques.

In [71]:
global_survival_fig = px.pie(
    train_data_filled,
    values=train_data_filled['Survived'].value_counts(),
    names=['Morts', 'Survivants'],
    title='Répartition de la survie sur le titanic'
)

global_survival_fig.show()

In [72]:
age_fig = px.box(
    train_data_filled,
    y='Age',
    title='Représentation de statistiques sur les âges'
)

age_fig.show()

In [73]:
age_violin_fig = px.violin(
    train_data_filled,
    y='Age',
    box=True,
    title='Distribution des âges des passagers (violin plot)'
)

age_violin_fig.show()

In [74]:
survival_by_title = train_data_filled.groupby('Title').agg({
    'Survived': ['count', 'sum']
})

survival_by_title.columns = ['Total', 'Survivors']
survival_by_title['Survival_Rate'] = (survival_by_title['Survivors'] / survival_by_title['Total'] * 100).round(2)

print(survival_by_title)

name_title_fig = px.bar(
    survival_by_title.reset_index(), # reset_index replace la colonne title correctement pour plotly
    x='Title',
    y='Survival_Rate',
    title='Pourcentage de survivants par titre',
    labels={
        'Survival_Rate': 'Taux de survie (%)',
        'Title': 'Titre'
    },
    text='Survival_Rate'
)

name_title_fig.show()

        Total  Survivors  Survival_Rate
Title                                  
Master     40         23          57.50
Miss      185        130          70.27
Mr        517         81          15.67
Mrs       126        100          79.37
Rare       23          8          34.78


In [75]:
survival_by_hascabin = train_data_filled.groupby('HasCabin').agg({
    'Survived': ['count', 'sum']
})

survival_by_hascabin.columns = ['Total', 'Survivors']
survival_by_hascabin['Survival_Rate'] = (survival_by_hascabin['Survivors'] / survival_by_hascabin['Total'] * 100).round(2)

print(survival_by_hascabin)

hascabin_fig = px.bar(
    survival_by_hascabin.reset_index(), # reset_index replace la colonne title correctement pour plotly
    x='HasCabin',
    y='Survival_Rate',
    title='Pourcentage de survivants s\'ils possèdent une cabine',
    labels={
        'Survival_Rate': 'Taux de survie (%)',
        'HasCabin': 'Possède ou non une cabine'
    },
    text='Survival_Rate'
)

hascabin_fig.update_xaxes(
    tickvals=[0,1],
    ticktext=['Sans cabine', 'Avec cabine']
)

hascabin_fig.show()

          Total  Survivors  Survival_Rate
HasCabin                                 
0           687        206          29.99
1           204        136          66.67


In [76]:
survival_by_cabindeck = train_data_filled.groupby('CabinDeck').agg({
    'Survived': ['count', 'sum']
})

survival_by_cabindeck.columns = ['Total', 'Survivors']
survival_by_cabindeck['Survival_Rate'] = (survival_by_cabindeck['Survivors'] / survival_by_cabindeck['Total'] * 100).round(2)

print(survival_by_cabindeck)

cabindeck_fig = px.bar(
    survival_by_cabindeck.reset_index(), # reset_index replace la colonne title correctement pour plotly
    x='CabinDeck',
    y='Survival_Rate',
    title='Pourcentage de survivants en fonction du pont de leur cabine',
    labels={
        'Survival_Rate': 'Taux de survie (%)',
        'CabinDeck': 'Possède ou non une cabine'
    },
    text='Survival_Rate'
)

cabindeck_fig.show()

           Total  Survivors  Survival_Rate
CabinDeck                                 
A             15          7          46.67
B             47         35          74.47
C             59         35          59.32
D             33         25          75.76
E             32         24          75.00
F             13          8          61.54
G              4          2          50.00
T              1          0           0.00
U            687        206          29.99


In [77]:
survival_by_isalone = train_data_filled.groupby('IsAlone').agg({
    'Survived': ['count', 'sum']
})

survival_by_isalone.columns = ['Total', 'Survivors']
survival_by_isalone['Survival_Rate'] = (survival_by_isalone['Survivors'] / survival_by_isalone['Total'] * 100).round(2)

print(survival_by_isalone)

isalone_fig = px.bar(
    survival_by_isalone.reset_index(), # reset_index replace la colonne title correctement pour plotly
    x='IsAlone',
    y='Survival_Rate',
    title='Pourcentage de survivants s\'ils sont seuls',
    labels={
        'Survival_Rate': 'Taux de survie (%)',
        'IsAlone': 'Seul(e) ou accompagné(e)'
    },
    text='Survival_Rate'
)

isalone_fig.update_xaxes(
    tickvals=[0,1],
    ticktext=['Accompagné(e)', 'Seul(e)']
)

isalone_fig.show()

         Total  Survivors  Survival_Rate
IsAlone                                 
0          354        179          50.56
1          537        163          30.35


In [78]:
survival_by_agebin = train_data_filled.groupby('AgeBin', observed=False).agg({
    'Survived': ['count', 'sum']
})

survival_by_agebin.columns = ['Total', 'Survivors']
survival_by_agebin['Survival_Rate'] = (survival_by_agebin['Survivors'] / survival_by_agebin['Total'] * 100).round(2)

print(survival_by_agebin)

agebin_fig = px.bar(
    survival_by_agebin.reset_index(), # reset_index replace la colonne title correctement pour plotly
    x='AgeBin',
    y='Survival_Rate',
    title='Pourcentage de survivants en fonction de la catégorie d\'âge',
    labels={
        'Survival_Rate': 'Taux de survie (%)',
        'AgeBin': 'Catégorie d\'âge'
    },
    text='Survival_Rate'
)

agebin_fig.show()

             Total  Survivors  Survival_Rate
AgeBin                                      
Child           77         45          58.44
Young_Adult    266        104          39.10
Adult          445        155          34.83
Middle_Aged     95         37          38.95
Senior           8          1          12.50


In [79]:
survival_by_farebin = train_data_filled.groupby('FareBin', observed=False).agg({
    'Survived': ['count', 'sum']
})

survival_by_farebin.columns = ['Total', 'Survivors']
survival_by_farebin['Survival_Rate'] = (survival_by_farebin['Survivors'] / survival_by_farebin['Total'] * 100).round(2)

print(survival_by_farebin)

farebin_fig = px.bar(
    survival_by_farebin.reset_index(), # reset_index replace la colonne title correctement pour plotly
    x='FareBin',
    y='Survival_Rate',
    title='Pourcentage de survivants en fonction de la tranche de prix du billet',
    labels={
        'Survival_Rate': 'Taux de survie (%)',
        'FareBin': 'Tranche de prix du billet'
    },
    text='Survival_Rate'
)

farebin_fig.show()

           Total  Survivors  Survival_Rate
FareBin                                   
Very_Low     226         51          22.57
Low          217         62          28.57
Medium       199         92          46.23
High         160         79          49.38
Very_High     74         57          77.03


Nous pouvons donc voir que le sexe et l'âge sont bien des facteurs majeurs de survie avec les taux exceptionnels des titres `Mrs`, `Miss` et `Master` là ou le taux est très bas pour le titre `Mr`.

La classe sociale est également un point à prendre en compte étant donné que le fait d'avoir une cabine garantissait `66,67 %` de chances de survie, les cabines situées sur les ponts `B`, `D` et `E` montaient jusqu'à `75 %`. Les personnes ayant payé les billets les plus chers ont survécu à `77,03 %`.

Les voyageurs accompagnés avaient en moyenne, plus de chances de survie que les voyageurs solitaires.

## 4. Modélisation

On a besoin de déterminer la survie des passagers avec une classification dans l'une de ces deux catégories : mort (0) et survivant (1).

In [80]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# variable cible > on va prédire qui a survécu 1 ou n'a pas survécu 0
y = train_data_filled['Survived'] 

# on définit quelles caractéristiques utiliser pour la prédiction
features = [
    'Sex',
    'AgeBin',
    'IsAlone',
    'HasCabin',
    'CabinDeck',
    'Title',
]

X = pd.get_dummies(train_data_filled[features])
X_test = pd.get_dummies(test_data_filled[features])

# aligner les colonnes entre train et test
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# diviser les données 80% pour apprendre 20% pour tester l'apprentissage
X_train, X_validation, y_train, y_validation = train_test_split(
    X, y,
    test_size=0.2,
    random_state=1
)

print(f"Groupe d'entraînement : {len(X_train)} passagers")
print(f"Groupe de validation  : {len(X_validation)} passagers")

# # voir si les proportions sont similaires
# taux_train = y_train.mean()
# taux_validation = y_validation.mean()

# print(f"entraînement: {taux_train:.1%}")
# print(f"Validation: {taux_validation:.1%}")

# normaliser les données
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)
X_test_scaled = scaler.transform(X_test)

sgd_model = SGDClassifier(
    loss='log_loss',
    max_iter=1000,
    random_state=42
)

sgd_model.fit(X_train_scaled, y_train)

predictions_validation = sgd_model.predict(X_validation_scaled)

precision = accuracy_score(y_validation, predictions_validation)

print(f"Précision sur la validation : {precision:.5%}")

predictions_test = sgd_model.predict(X_test_scaled)

output = pd.DataFrame({
    'PassengerId': test_data_filled.PassengerId,
    'Survived': predictions_test
})

output.to_csv('submissions/submission6.csv', index=False)

Groupe d'entraînement : 712 passagers
Groupe de validation  : 179 passagers
Précision sur la validation : 77.65363%


In [81]:
# from sklearn.ensemble import RandomForestClassifier

# # variable cible > on va prédire qui a survécu 1 ou n'a pas survécu 0
# y = train_data_filled['Survived'] 

# # on définit quelles caractéristiques utiliser pour la prédiction
# features = [
#     'Sex',
#     'AgeBin',
#     'IsAlone',
#     'HasCabin',
#     'CabinDeck',
#     'Title',
# ]

# X = pd.get_dummies(train_data_filled[features])
# X_test = pd.get_dummies(test_data_filled[features])

# # aligner les colonnes entre train et test
# X_test = X_test.reindex(columns=X.columns, fill_value=0)

# model = RandomForestClassifier(
#     n_estimators=100, # nombre d'arbres de la forêt > + = plus précis mais plus lent
#     max_depth=5, # profondeur max par arbre > évite l'overfitting (quand le modèle s'adapte trop aux données d'entrainement)
#     random_state=42 # comme une seed > pour pas que cela ne soit random à chaque fois
# )

# # entrainement du modèle
# model.fit(X, y)

# # mise en application
# predictions = model.predict(X_test)

# output = pd.DataFrame({
#     'PassengerId': test_data_filled.PassengerId,
#     'Survived': predictions # les prédictions 0 ou 1
# })

# output.to_csv('submissions/submission5.csv', index=False)

# print('success')